# Group 2 Final Project Technical Notebook

Name(s): Dominic Fanucchi, Carlo Sanchez, Hani Jandali    
Date:    November 5, 2023

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns


# Style sheet to use with Matplotlib and Seaborn plots
plt.style.use('ggplot')
pd.set_option('display.max_columns', 200)